# Neighborhoods in Toronto

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

    <ol> 
        <li><a href="#1.-Toronto-Neighborhoods-data">Toronto Neighborhoods data</a></li>
        <li><a href="#2.-Geo-data-for-the-neighborhoods">Geo data for the neighborhoods</a></li>
        <li><a href="#3.-Exploring-and-Clusterization">Exploring and Clusterization</a></li>     
    </ol>
</div>

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import os

from sklearn.cluster import KMeans
import folium

## 1. Toronto Neighborhoods data

Scrapping Wiki page.
And parsing the grabbed HTML with BeautifulSoup and simple selector

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

wiki_data = soup.select("#mw-content-text table.wikitable td")
wiki_data = [x.text for x in wiki_data]

Creating pandas dataframe.

In [7]:
data = {
    "PostalCode": wiki_data[0::3], 
    "Borough": wiki_data[1::3],
    "Neighborhood": wiki_data[2::3]
}

# dataframe preserving order
df = pd.DataFrame(data, columns=["PostalCode", "Borough", "Neighborhood"])

Data peparation. Some garbage cleanup. Removing "Not assigned" rows etc.

In [8]:
# data cleanup
garbage_patterns = [
    "\\n", # scrapped html artefacts
    "\]"   # one more ugly char
]
df["Neighborhood"] = df["Neighborhood"].replace(to_replace=garbage_patterns, value="", regex=True)

# "Not assigned" garbage cleanup
# replacing it Panda way, with np.nan ("NaN")
df = df.replace("Not assigned", np.nan)

# removing rows with two "Not assigned" values
df = df.dropna(thresh=2)

# replacing empty Neighborhood with Borough value
df["Neighborhood"] = df["Neighborhood"].fillna(df["Borough"])

# and final reindex, coz we don't need old indexes
df = df.reset_index(drop=True)

print(df.shape)
df.head()

(212, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Grouping and concatenating of Neighboorhood groups

In [9]:
# grouping and concatenating of Neighboorhood groups
toronto_neighborhoods = df.groupby(["PostalCode", "Borough"], as_index=False)\
.Neighborhood.agg({"Neighborhood": lambda x: ", ".join(list(x))})


In [10]:
toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
toronto_neighborhoods.shape

(103, 3)

## 2. Geo data for the neighborhoods

In [12]:
# cache for geodata
# solving problem with geocoder queries limit
latlng_cache = {}

In [13]:
def get_latlng(postal_code):
    
    query = '{}, Toronto, Ontario'.format(postal_code)
    
    # try to get result from cache first
    if query in latlng_cache:
        lat_lng_coords = latlng_cache[query]
    else:
        # initialize your variable to None
        lat_lng_coords = None

        # loop until you get the coordinates
        while(lat_lng_coords is None):

            g = geocoder.google(query)
            latlng_cache[query] = g.latlng
            lat_lng_coords = latlng_cache[query]
    
    return lat_lng_coords[0], lat_lng_coords[1]

In [14]:
# geocoder.google('Mountain View, CA')
# latlng_cache

In [15]:
latlng = toronto_neighborhoods.apply(lambda x: get_latlng(x["PostalCode"]), axis=1)

toronto_neighborhoods["Latitude"] = latlng.apply(lambda x: x[0])
toronto_neighborhoods["Longitude"] = latlng.apply(lambda x: x[1])


In [17]:
toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
toronto_neighborhoods.shape

(103, 5)

## 3. Exploring and Clusterization

### Exploring the data

Getting Foursquare API credentials from env variables

In [19]:
CLIENT_ID = os.environ.get("FOURSQUARE_CLIENT_ID")
CLIENT_SECRET = os.environ.get("FOURSQUARE_CLIENT_SECRET")
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighborhood'],
                                   latitudes=toronto_neighborhoods['Latitude'],
                                   longitudes=toronto_neighborhoods['Longitude']
                                  )



Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(2232, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Restaurant
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [26]:
# one hot encoding
toronto_1hot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_1hot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = list(toronto_1hot.columns)
cols.remove("Neighborhood")
reordered_cols = ['Neighborhood'] + cols
toronto_1hot = toronto_1hot[reordered_cols]

toronto_1hot.head()


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_grouped = toronto_1hot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.010000,0.000000,0.000000,0.000000
1,Agincourt,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.052632,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


Filter original DataFrame.
Removing neighborhoods where FourSquare didn't find info

In [31]:
# filter original DataFrame 
# removing neighborhoods where FourSquare didn't find info
toronto_neighborhoods_filtered = toronto_neighborhoods.loc[
    toronto_neighborhoods["Neighborhood"].isin(toronto_grouped["Neighborhood"].values)
].reset_index(drop=True)
toronto_neighborhoods_filtered.shape

(100, 5)

### Clustering Toronto neighborhoods

In [33]:
# set number of clusters
kclusters = 5

grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])


[0 0 2 0 0 0 0 0 0 0]


In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Agincourt----
                venue  freq
0        Skating Rink  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3      Clothing Store  0.25
4  Mexican Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                        venue  freq
0                  Playground   0.5
1                        Park   0.5
2           Accessories Store   0.0
3                 Men's Store   0.0
4  Modern European Restaurant   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0          Pizza Place   0.2
1        Grocery Store   0.2
2           Beer Store   0.1
3  Fried Chicken Joint   0.1
4       Sandwich Place   0.1


----Alderwood, Long Branch

                venue  freq
0      Discount Store  0.29
1    Department Store  0.14
2          Hobby Shop  0.14
3         Coffee Shop  0.14
4  Chinese Restaurant  0.14


----East Toronto----
                             venue  freq
0                             Park  0.67
1                Convenience Store  0.33
2                Accessories Store  0.00
3                    Metro Station  0.00
4  Molecular Gastronomy Restaurant  0.00


----Emery, Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1                Accessories Store   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.09
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.06
3                Bakery  0.05
4      Asian Restaurant  0.05


----First Canadian Place, Underground city----
                 venue

                venue  freq
0                Café  0.08
1         Coffee Shop  0.08
2  Italian Restaurant  0.06
3         Pizza Place  0.06
4    Sushi Restaurant  0.06


----Ryerson, Garden District----
                 venue  freq
0          Coffee Shop  0.08
1       Clothing Store  0.07
2       Cosmetics Shop  0.04
3                 Café  0.04
4  Japanese Restaurant  0.03


----Scarborough Village----
                        venue  freq
0                  Playground   1.0
1           Accessories Store   0.0
2                 Men's Store   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0


----Silver Hills, York Mills----
                             venue  freq
0                Martial Arts Dojo   1.0
1                Accessories Store   0.0
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.08
1      Restaurant  0.06

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Bakery,Hotel,Bar,Burger Joint,Restaurant
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Pharmacy,Fried Chicken Joint,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place,Yoga Studio,Discount Store
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pub,Sandwich Place,Bank,Gym,Airport Service,Farmers Market,Event Space
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Grocery Store,Chinese Restaurant,Ice Cream Shop,Sushi Restaurant,Deli / Bodega,Bank,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
6,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
7,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Juice Bar,Indian Restaurant,Restaurant,Butcher,Pub,Liquor Store
8,Berczy Park,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Farmers Market,Beer Bar,Cheese Shop,Steakhouse,Bakery,Restaurant
9,"Birch Cliff, Cliffside West",Café,College Stadium,Skating Rink,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [38]:
toronto_merged = toronto_neighborhoods_filtered


# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Medical Center,Electronics Store,Restaurant,Rental Car Location,Breakfast Spot,Mexican Restaurant,Doner Restaurant,Diner,Discount Store,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Cosmetics Shop,Coworking Space,Event Space


In [55]:
address = 'Toronto, Canada'

location = geocoder.google(address).latlng
latitude = location[0]
longitude = location[1]
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653226, -79.3831843.


In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Clusterization summary

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
1,"Highland Creek, Rouge Hill, Port Union",0,Bar,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Fast Food Restaurant
3,Woburn,0,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,Cedarbrae,0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Cosmetics Shop,Coworking Space,Event Space
5,Scarborough Village,0,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
6,"East Birchmount Park, Ionview, Kennedy Park",0,Discount Store,Train Station,Chinese Restaurant,Coffee Shop,Hobby Shop,Department Store,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,"Clairlea, Golden Mile, Oakridge",0,Bakery,Soccer Field,Park,Bus Station,Bus Line,Metro Station,Ice Cream Shop,Dog Run,Diner,Discount Store
8,"Cliffcrest, Cliffside, Scarborough Village West",0,American Restaurant,Motel,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
9,"Birch Cliff, Cliffside West",0,Café,College Stadium,Skating Rink,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,"Dorset Park, Scarborough Town Centre, Wexford ...",0,Indian Restaurant,Chinese Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Hillcrest Village,1,Mediterranean Restaurant,Dog Run,Golf Course,Pool,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop
72,Caledonia-Fairbanks,1,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
76,"Brockton, Exhibition Place, Parkdale Village",1,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Furniture / Home Store,Performing Arts Venue,Pet Store,Gym,Nightclub,Convenience Store
99,Northwest,1,Bar,Rental Car Location,Drugstore,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",2,Medical Center,Electronics Store,Restaurant,Rental Car Location,Breakfast Spot,Mexican Restaurant,Doner Restaurant,Diner,Discount Store,Dog Run
40,"The Beaches West, India Bazaar",2,Park,Fast Food Restaurant,Burger Joint,Ice Cream Shop,Liquor Store,Steakhouse,Fish & Chips Shop,Pub,Italian Restaurant,Burrito Place
74,"Dovercourt Village, Dufferin",2,Bakery,Supermarket,Gym,Pharmacy,Pizza Place,Music Venue,Middle Eastern Restaurant,Café,Discount Store,Brewery
79,"The Junction North, Runnymede",2,Convenience Store,Pizza Place,Bakery,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
93,Humber Summit,2,Pizza Place,Empanada Restaurant,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Studio District,3,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Seafood Restaurant,Fish Market,New American Restaurant
55,Central Bay Street,3,Coffee Shop,Italian Restaurant,Café,Japanese Restaurant,Burger Joint,Bar,Chinese Restaurant,Thai Restaurant,Sushi Restaurant,Sandwich Place


In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,"Ryerson, Garden District",4,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Bar,Japanese Restaurant,Italian Restaurant,Theater,Middle Eastern Restaurant,Movie Theater
